In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('final_data.csv')

In [ ]:
df.head(5)

,user,time,lat,lon,city,state,text
0,USER_79321756,2010-03-03T04:15:26,47.528139,-122.197916,Renton,Washington,RT @USER_2ff4faca: IF SHE DO IT 1 MORE TIME......
1,USER_79321756,2010-03-03T04:55:32,47.528139,-122.197916,Renton,Washington,@USER_77a4822d @USER_2ff4faca okay:) lol. Sayi...
2,USER_79321756,2010-03-03T05:13:34,47.528139,-122.197916,Renton,Washington,RT @USER_5d4d777a: YOURE A FAG FOR GETTING IN ...
3,USER_79321756,2010-03-03T05:28:02,47.528139,-122.197916,Renton,Washington,@USER_77a4822d yea ok..well answer that cheap ...
4,USER_79321756,2010-03-03T05:56:13,47.528139,-122.197916,Renton,Washington,A sprite can disappear in her mouth - lil kim ...


In [ ]:
df.shape

(377610, 7)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377610 entries, 0 to 377609
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   user    377610 non-null  object 
 1   time    377610 non-null  object 
 2   lat     377610 non-null  float64
 3   lon     377610 non-null  float64
 4   city    350658 non-null  object 
 5   state   375083 non-null  object 
 6   text    377560 non-null  object 
dtypes: float64(2), object(5)
memory usage: 20.2+ MB


In [ ]:
df.dropna(subset=['text'], inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 377560 entries, 0 to 377609
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   user    377560 non-null  object 
 1   time    377560 non-null  object 
 2   lat     377560 non-null  float64
 3   lon     377560 non-null  float64
 4   city    350611 non-null  object 
 5   state   375034 non-null  object 
 6   text    377560 non-null  object 
dtypes: float64(2), object(5)
memory usage: 23.0+ MB


In [ ]:
df.shape

(377560, 7)

In [ ]:
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()

In [ ]:
def steam(text):
   y=[]
   for i in text.split():
         y.append(ps.stem(i))
   return " ".join(y)

In [ ]:
df['text']=df['text'].apply(steam)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv= CountVectorizer(max_features=500, stop_words='english')
vectors=cv.fit_transform(df['text']).toarray()

In [ ]:
vectors.shape

(348208, 500)

In [ ]:
df['vectors']=''
cnt=0
for i in vectors:
    df['vectors'][cnt]=i
    cnt+=1

In [ ]:
df.user.unique()

array(['USER_79321756', 'USER_6197f95d', 'USER_ce270acf', ...,
       'USER_10ff1222', 'USER_28bf1046', 'USER_8c704efa'], dtype=object)

In [ ]:
import numpy as np
from numpy.linalg import norm

In [ ]:
def recommendation(user_id,recent_cnt):
  recent_tweet_vector=df[df['user']==user_id].sort_values(by='time',ascending=False).vectors.iloc[recent_cnt-1]
  recent_tweet=df[df['user']==user_id].sort_values(by='time',ascending=False).text.iloc[recent_cnt-1]
  similarity=[]
  for i in df.index:
    if df['user'][i]!=user_id:
        try:
            vector_i = np.array(df['vectors'][i], dtype=float)  # Convert to float
        except ValueError:
            continue
        # Calculate similarity if both vectors are valid
        if recent_tweet_vector.size > 0 and vector_i.size > 0:
            similarity.append([np.dot(recent_tweet_vector, vector_i) / (norm(recent_tweet_vector) * norm(vector_i)), i,df['text'][i]])  # Use index 'i' instead of 'id'
  similarity.sort(key=lambda x: x[0],reverse=True)
  return recent_tweet,similarity[0:10]

In [ ]:
tweet,lst= recommendation('USER_ce270acf',1)
print('original_text:'+ tweet,end='\n\n')
for i in lst:
    print('similarity'+ str(i[0]))
    print('text'+ str(i[2]))
    print('\n\n')

<ipython-input-114-b82ecb9f36bb>:13: RuntimeWarning: invalid value encountered in scalar divide
  similarity.append([np.dot(recent_tweet_vector, vector_i) / (norm(recent_tweet_vector) * norm(vector_i)), i,df['text'][i]])  # Use index 'i' instead of 'id'


original_text:@user_0544edb9 lol shut up

similarity0.7071067811865475
text@user_621b922f thanks:). lol.



similarity0.7071067811865475
text@user_f67b3c69 hahahahaha



similarity0.7071067811865475
text@user_0e7677c1 no i wa here ju wasn't chargin my phone ...i'm mad now lol



similarity0.4999999999999999
text@user_02ad3563 guess i'll have to wait up for thi stori lol



similarity0.4999999999999999
textgot p' in my head but don't call me a p-head cuz it a p-hawk lol



similarity0.4714045207910316
text"rt @user_620cd4b9: @user_79321756 hey now! leav me, and my big eye alon lol>>lol no! :*"



similarity0.42640143271122083
textrt @user_dc5e5498: @user_79321756 so your tri to say im pressed. #supertweet ?? lmao hahah>>lol. i'd rather get em on twitter only. lol.



similarity0.42640143271122083
textrt @user_7ec2204e: @user_79321756 i didn't say anyth about cake...? lol>>lol but she make one err week so i know there' one! duh!



similarity0.40824829046386296
textrt @user_3914673e: i h

In [ ]:
tweet,lst= recommendation('USER_79321756',7)
print('original_text:'+ tweet,end='\n\n')
for i in lst:
    print('similarity'+ str(i[0]))
    print('text'+ str(i[2]))
    print('\n\n')

<ipython-input-114-b82ecb9f36bb>:13: RuntimeWarning: invalid value encountered in scalar divide
  similarity.append([np.dot(recent_tweet_vector, vector_i) / (norm(recent_tweet_vector) * norm(vector_i)), i,df['text'][i]])  # Use index 'i' instead of 'id'


original_text:i should go get some henny! but ima chill out:).

similarity0.40824829046386296
text@user_b71ed7f4 naw i dont...lmao.



similarity0.0
textwatch lost



similarity0.0
text@user_89a3500b i did



similarity0.0
text"maneuv so that i can put my team on, hope sooner so that we can live our dream on"



similarity0.0
textdarko wa eat hamburg in the locker room befor they play the knicks. lol



similarity0.0
textgirl pack ya bag i'm bout to take you on a ride!



similarity0.0
text"@user_a9cf8f82 lol, yeah check it out bro"



similarity0.0
text"rt @user_5eae722d: #inhighschool me & mr. staviski dnt lk each other, hi breath smell lk straight ass! - lmao he use 2 chase us dn the hall"



similarity0.0
text#inhighschool trenton high girl basketbal team alway had the best record out of all the highschool sport teams. noth ha chang



similarity0.0
textthe cheerlead team need mrs. gradi back



